Restructure Dataset

In [ ]:
import os
import shutil
from pathlib import Path

original_path = Path("Skin_Data")
new_base = Path("restructured_data")
train_path = new_base / "train"
val_path = new_base / "val"

# Create new directories
for cls in ["Cancer", "Non_Cancer"]:
    os.makedirs(train_path / cls, exist_ok=True)
    os.makedirs(val_path / cls, exist_ok=True)
    
    # Move training images
    train_imgs = list((original_path / cls / "Training").glob("*.JPG"))
    for img in train_imgs:
        shutil.copy(img, train_path / cls / img.name)

    # Move testing images
    test_imgs = list((original_path / cls / "Testing").glob("*.JPG"))
    for img in test_imgs:
        shutil.copy(img, val_path / cls / img.name)

print("✅ Dataset Restructured!")


Image Generators for Training and Validation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    directory=str(train_path),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_gen = datagen.flow_from_directory(
    directory=str(val_path),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


Build the CNN Model (Using MobileNetV2)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

base_model = MobileNetV2(input_shape=img_size + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Train the Model

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

Visualize Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


Predict on a New Image

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

def predict_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0  # Normalize

    prediction = model.predict(img_array)[0][0]
    confidence = prediction * 100

    # Show image
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {'Cancer' if prediction > 0.5 else 'Non-Cancer'}\nConfidence: {confidence:.2f}%")
    plt.show()

    print(f"🔍 Prediction Confidence: {confidence:.2f}%")
    print("⚠️ Malignant (Cancer)" if prediction > 0.5 else "✅ Benign (Non-Cancer)")

# Example:
predict_image("test.jpg")  # Replace with your test image

Predict on a New Image

In [ ]:
model.save("skin_cancer_model.h5")
print("💾 Model saved as skin_cancer_model.h5")